In [2]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.utils import to_categorical

# Load images and labels from directory
def load_images_with_labels(base_directory):
    images = []
    labels = []
    processed_counts = {}
    label_map = {'an2i': 0, 'at33': 1, 'boland': 2, 'bpm': 3, 'ch4f' : 4, 'cheyer' : 5, 'choon' : 6, 'danieln' : 7, 'glickman' : 8, 'karyadi' : 9, 'kawamura' : 10, 'kk49' : 11, 'megak' : 12, 'mitchell' : 13, 'night' : 14, 'phoebe' : 15, 'saavik' : 16, 'steffi' : 17, 'sz24' : 18, 'tammo' : 19}
    
    for root, dirs, files in os.walk(base_directory):
        folder_name = os.path.basename(root)
        processed_counts[folder_name] = {'total': 0, 'processed': 0, 'skipped': 0}
        
        for filename in files:
            if filename.endswith(('.png', '.pgm')):
                file_path = os.path.join(root, filename)
                try:
                    processed_counts[folder_name]['total'] += 1
                    
                    # Load and preprocess image
                    img = Image.open(file_path).convert('RGB')
                    img = img.resize((64, 64))
                    img_array = np.array(img) / 255.0
                    
                    # Parse label from filename
                    parts = filename.split('_')
                    if len(parts) > 1:
                        orientation = parts[0]
                        if orientation in label_map:
                            images.append(img_array)
                            labels.append(label_map[orientation])
                            processed_counts[folder_name]['processed'] += 1
                    else:
                        processed_counts[folder_name]['skipped'] += 1
                        print(f"Skipping file with missing orientation: {filename} in {folder_name}")
                        
                except Exception as e:
                    processed_counts[folder_name]['skipped'] += 1
                    print(f"Error processing {filename} in {folder_name}: {e}")
    
    return np.array(images), np.array(labels)

# Define the combined model with autoencoder and classifier
def create_combined_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder to reconstruct the image
    x = UpSampling2D((2, 2))(encoded)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same', name='decoded')(x)

    # Classification branch for orientation prediction
    x = GlobalAveragePooling2D()(encoded)
    x = Dense(64, activation='relu')(x)  # Added Dense layer
    classification_output = Dense(num_classes, activation='softmax', name='classification')(x)

    # Build model
    model = Model(inputs, [decoded, classification_output])
    model.compile(
        optimizer='adam',
        loss={
            'decoded': 'mean_squared_error',
            'classification': 'categorical_crossentropy'
        },
        loss_weights={
            'decoded': 0.5,
            'classification': 1.0
        },
        metrics={
            'classification': ['accuracy']
        }
    )

    return model

# Main script to load data, train and evaluate the model
directory = 'faces'  # specify data directory
images, labels = load_images_with_labels(directory)

# Ensure we have data to proceed
if len(images) == 0 or len(labels) == 0:
    print("No data found. Ensure the directory is correct and files match the expected format.")
else:
    # Train/test split and categorical encoding of labels
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    y_train_cat = to_categorical(y_train, num_classes=20)
    y_test_cat = to_categorical(y_test, num_classes=20)

    # Instantiate and train the model
    combined_model = create_combined_model(input_shape=(64, 64, 3), num_classes=20)
    combined_model.fit(
        X_train,
        {
            'decoded': X_train,             # For the autoencoder output
            'classification': y_train_cat   # For the classification output
        },
        batch_size=64,
        epochs=15,
        validation_data=(
            X_test,
            {
                'decoded': X_test,
                'classification': y_test_cat
            }
        )
    )

    # Make predictions on test data
    decoded_imgs, predictions = combined_model.predict(X_test)
    predicted_labels = np.argmax(predictions, axis=1)

    # Map numeric labels back to string labels for readability
    orientation_map = {0: 'an2i', 1: 'at33', 2: 'boland', 3: 'bpm', 4: 'ch4f', 5: 'cheyer', 6: 'choon', 7: 'danieln', 8: 'glickman',9: 'karyadi',10: 'kawamura', 11: 'kk49',12: 'megak',13: 'mitchell', 14: 'night', 15: 'phoebe',16: 'saavik',17: 'steffi',18: 'sz24',19: 'tammo'}
    correct, wrong = 0, 0

    # Print predictions for comparison
    for i in range(len(y_test)):
        actual = orientation_map[y_test[i]]
        predicted = orientation_map[predicted_labels[i]]
        print(f"Actual: {actual}, Predicted: {predicted}")
        if actual == predicted:
            correct += 1
        else:
            wrong += 1
    print(f"Total Correct: {correct}")
    print(f"Total Wrong: {wrong}")
     

Epoch 1/15
467/467 ━━━━━━━━━━━━━━━━━━━━ 105s 216ms/step - classification_accuracy: 0.1322 - loss: 2.7304 - val_classification_accuracy: 0.5166 - val_loss: 1.4850
Epoch 2/15
467/467 ━━━━━━━━━━━━━━━━━━━━ 98s 209ms/step - classification_accuracy: 0.5683 - loss: 1.3175 - val_classification_accuracy: 0.7614 - val_loss: 0.8010
Epoch 3/15
467/467 ━━━━━━━━━━━━━━━━━━━━ 103s 220ms/step - classification_accuracy: 0.7632 - loss: 0.7528 - val_classification_accuracy: 0.8044 - val_loss: 0.6025
Epoch 4/15
467/467 ━━━━━━━━━━━━━━━━━━━━ 97s 209ms/step - classification_accuracy: 0.8695 - loss: 0.4421 - val_classification_accuracy: 0.8973 - val_loss: 0.3320
Epoch 5/15
467/467 ━━━━━━━━━━━━━━━━━━━━ 98s 210ms/step - classification_accuracy: 0.9076 - loss: 0.2930 - val_classification_accuracy: 0.9436 - val_loss: 0.1946
Epoch 6/15
467/467 ━━━━━━━━━━━━━━━━━━━━ 96s 205ms/step - classification_accuracy: 0.9417 - loss: 0.1890 - val_classification_accuracy: 0.9493 - val_loss: 0.1650
Epoch 7/15
467/467 ━━━━━━━━━━━━━